In [ ]:
!pip install camel-tools==1.2.0 farasapy nltk Arabic-Stopwords scikit-learn gensim pandas
!camel_data light
import nltk
nltk.download('stopwords')

In [ ]:
# built in modules
import numpy as np
import pandas as pd
# from sklearn import metrics 

# .py files 
from preprocess import Preprocess
# from feature_extraction import *

from classical_models import LSVMmodel
# Can use this library to reload a specific module if the notebook can't see changes in the imported module
import importlib

Stance detection labels meaning is as follows:
1. Positive (1): means that the tweet author encourages and supports vaccination.
2. Negative (-1): means that the tweet author refuses vaccination.
3. Neutral (0): means that the tweet neither supports nor refuses vaccination.

Category labels meaning is as follows:
1. Info_News: Information about vaccination.
2. Celebrities: mentioning celebrities taking vaccinations.
3. Plan: Governmental plan or progress of vaccination.
4. Request: Requests from governments regarding the vaccination process.
5. Rumor: the tweet is a rumor.
6. Advice: Advice related to the virus or the vaccination
7. Restriction: Restrictions due to the virus e.g. traveling.
8. Personal: Personal opinion or story about vaccination.
9. Unrelated: Unrelated to vaccination.
10.Others: Vaccination related but not one of the above.

In [ ]:
# !unzip Dataset.zip
t = pd.read_csv('./Dataset/train.csv')
d = pd.read_csv('./Dataset/dev.csv')
ts = pd.read_csv('./Dataset/test.csv')
#
# ov = pd.read_csv('/content/Dataset/classification_train_sample1.csv')
# ov2 = pd.read_csv('/content/Dataset/cat_somesample.csv')

In [ ]:
# categories = ["info_news", "celebrity", "plan", "requests", "rumors", "advice", "restrictions", "personal", "unrelated", "others"]
# def encode_category(y):
#     '''
#     Input: y a list of string labels for the category of each document 
#     Output: a list of encoded 10 sized array for the category of each doc 
#             for "other" category , it has an array =[0 0 0 0 0 0 0 0 0 1] 
#     '''
#     return [categories.index(ele) for ele in y]

### Understanding the Data
- 80% tweets are positive (class 1), the rest are neutral and negative.
- 50% tweets belongs to info_news category.

In [ ]:
# # analyze dataset
# print("counts for each lablel :")
# print(t['category'].value_counts(normalize=True))
# print(ov['category'].value_counts(normalize=True))
# print(ov2['category'].value_counts(normalize=True))
# print(d['category'].value_counts(normalize=True))

In [ ]:
# print("####################################")
# print("duplicated rows :")
# print(t[t.duplicated()])

In [ ]:
X_train = t['text']
# X_trainov = ov['text']
# X_trainov2 = ov2['text']
X_dev = d['text']
X_test = ts['text']
#
Y_train = t['category']
# Y_trainov = ov['category']
# Y_trainov2 = ov2['category']
Y_dev = d['category']
# Y_test = ts['text']

#### Preprocessing

In [ ]:
preprocess = Preprocess()
X_train = X_train.apply(preprocess.do_all)
# X_trainov = X_trainov.apply(preprocess.do_all)
# X_trainov2 = X_trainov2.apply(preprocess.do_all)
X_dev = X_dev.apply(preprocess.do_all)
# X_test = X_test.apply(preprocess.do_all)

In [ ]:
# r = random.randint(0, len(X_test))
# # r = 900
# print(r, '\n')
# print(ts.text[r], '\n')
# print(X_test[r], '\n')

In [ ]:
# # Print Vocabulary size
# lst = [word for x in X_train for word in x]
# vocab = set(lst)
# print(len(lst))
# print(len(vocab))

### Getting the train , test and dev features

In [ ]:
# train_features_bow,test_features_bow=BOW(train_documents=X,test_documents=X_dev)
# #
# assert train_features_bow.shape[0] == len(X)
# assert train_features_bow.shape[1] == len(vocab)

In [ ]:
# train_features_tfidf,test_features_tfidf=TFIDF(train_documents=X,test_documents=X_dev)
# #
# assert train_features_tfidf.shape[0] == len(X)
# assert train_features_tfidf.shape[1] == len(vocab)

In [ ]:
# from sklearn.feature_extraction.text import TfidfVectorizer

# vectorizer =TfidfVectorizer(analyzer=lambda x: x)
# train_features= vectorizer.fit_transform(X)
# # return train_features.toarray(), test_features.toarray()

# # print 10 words with highest tfidf values for each CATEGORY
# feature_names = vectorizer.get_feature_names_out()
# for i, category in enumerate(categories):
#     lst = [0] *len(feature_names)
#     for row in train_features[np.array(yc) == i].toarray():
#         lst = [x + y for x, y in zip(lst, row)]
#     # top 10 words
#     top20 = np.argsort(lst)[::-1][:20]
#     # print names and values
#     print(category)
#     print([(feature_names[j], lst[j]) for j in top20])
#     print()

In [ ]:
# train_features_cbow,test_features_cbow=CBOW(train_documents=X,test_documents=X_dev)
# #
# print(train_features_cbow.shape)
# print(X.shape)

In [ ]:
# train_features_sg,test_features_sg=SG(train_documents=X,test_documents=X_dev)
# #
# print(train_features_sg.shape)

## Classical model Training 

- Classification Task Results:

|  | SVM | Random Forest | KNN |  Voting system |
| --------------- | --------------- | --------------- | --------------- | --------------- |
| TFIDF |69%  |  |  ||
| BOW | 54%  |   |   | |


- Stance Task Results:

|  | SVM | Random Forest | KNN |  Voting system |
| --------------- | --------------- | --------------- | --------------- | --------------- |
| TFIDF |81%  |  |  ||
| BOW | 80%  |   |   | |

In [ ]:
# train_features = np.concatenate((train_features_bow, train_features_tfidf, train_features_cbow, train_features_sg), axis=1)
# test_features = np.concatenate((test_features_bow, test_features_tfidf, test_features_cbow, test_features_sg), axis=1)
# #
# print(train_features_bow.shape)
# print(train_features.shape)

In [ ]:
#  clf=LogisticRegression(max_iter=200, class_weight={0:1,1:2,2:3,3:4,4:1000,5:5,6:15,7:2,8:4,9:100000}
from sklearn.feature_extraction.text import TfidfVectorizer
def TFIDFW(train_documents_,test_documents_):
    # vectorizer =TfidfVectorizer(analyzer=lambda x: x, ngram_range=(10, 10))
    vectorizer =TfidfVectorizer(token_pattern=r'\S+', lowercase=False, tokenizer=lambda x: x, ngram_range=(1, 3))
    # vectorizer =TfidfVectorizer(analyzer='char', ngram_range=(4, 4))

    train_documents = [" ".join(doc) for doc in train_documents_]
    test_documents = [" ".join(doc) for doc in test_documents_]
    train_features= vectorizer.fit_transform(train_documents)
    test_features=vectorizer.transform(raw_documents=test_documents)
    return train_features.toarray(), test_features.toarray()

def TFIDFC(train_documents_,test_documents_):
    # vectorizer =TfidfVectorizer(analyzer=lambda x: x, ngram_range=(10, 10))
    # vectorizer =TfidfVectorizer(token_pattern=r'\S+', lowercase=False, tokenizer=lambda x: x, ngram_range=(1, 4))
    vectorizer =TfidfVectorizer(analyzer='char', ngram_range=(2, 3))

    train_documents = [" ".join(doc) for doc in train_documents_]
    test_documents = [" ".join(doc) for doc in test_documents_]
    train_features= vectorizer.fit_transform(train_documents)
    test_features=vectorizer.transform(raw_documents=test_documents)
    return train_features.toarray(), test_features.toarray()

def TFIDF(tr,ts):
    trw, tsw = TFIDFW(tr, ts)
    trc, tsc = TFIDFC(tr, ts)
    return np.concatenate((trw, trc), axis=1), np.concatenate((tsw, tsc), axis=1)

In [ ]:
# train_features, test_features = TFIDF(X_train, X_dev)

# from sklearn import metrics

# # clss = [NBmodel, KNNmodel, SVMmodel, LRmodel, RGmodel, RFmodel, DTmodel]
# # clss = [KNNmodel]
# clss = [LSVMmodel]
# #
# for cls in clss:
#     #
#     print(cls.__name__)
#     yc_pred = cls(Xtrain=train_features, y_train=Y_train, X_test=test_features)
#     # ys_pred = cls(Xtrain=train_features, y_train=Y_train['stance'], X_test=test_features)
#     #
#     print('========= Category =========')
#     print(metrics.classification_report(y_true=Y_dev,y_pred=yc_pred))
#     print('========= Stance =========')
#     # print(metrics.classification_report(y_true=Y_dev['stance'],y_pred=ys_pred))

# # classify_accuracy=metrics.accuracy_score(y_true=yc_dev,y_pred=yc_pred)
# # stance_accuracy=metrics.accuracy_score(y_true=ys_dev,y_pred=ys_pred)

In [ ]:
# get output of test set
X_train_dev = np.concatenate((X_train, X_dev), axis=0)
Y_train_dev = np.concatenate((Y_train, Y_dev), axis=0)
print(X_train.shape, X_dev.shape, X_train_dev.shape)
print(Y_train_dev.shape)
train_features, test_features = TFIDF(X_train_dev, X_test)

yc_pred = LSVMmodel(Xtrain=train_features, y_train=Y_train_dev, X_test=test_features)

In [ ]:
# yc_pred
pred = [(i, vi) for i, vi in enumerate(yc_pred)]

print(pred[:15])
# print(pred.value_counts())
# print(metrics.classification_report(labels, pred))

In [ ]:
from sklearn import svm
clf=svm.LinearSVC(class_weight='balanced')
clf.fit(X=train_features,y=Y_train_dev)

from joblib import dump, load
dump(clf, 'filename.joblib') 

In [ ]:
import csv
with open('./out/test_cat.csv', "w",encoding="utf-8", newline='') as f:
    writer = csv.writer(f)
    writer.writerow(('id','stance'))
    for row in pred:
        writer.writerow(row)

dd = pd.read_csv('./out/test_cat.csv')
print(dd.head())

print(dd.stance.value_counts())

In [ ]:
asds

**SVM**

In [ ]:
yc_pred=SVMmodel(Xtrain=train_features_bow,y_train=yc,X_test=test_features_bow)

In [ ]:
ys_pred=SVMmodel(Xtrain=train_features_tfidf,y_train=ys,X_test=test_features_tfidf)

In [ ]:
yc_cbow_pred=SVMmodel(Xtrain=train_features_cbow,y_train=yc,X_test=test_features_cbow)

In [ ]:
ys_cbow_pred=SVMmodel(Xtrain=train_features_cbow,y_train=ys,X_test=test_features_cbow)

In [ ]:
classify_accuracy=metrics.accuracy_score(y_true=yc_dev,y_pred=yc_cbow_pred)
stance_accuracy=metrics.accuracy_score(y_true=ys_dev,y_pred=ys_cbow_pred)


print("The classification accuracy after training on svm on all features : ",classify_accuracy)
print("The stance accuracy after training on svm on all features : ",stance_accuracy)

# F1 score
print('========= Category =========')
print(metrics.classification_report(y_true=yc_dev,y_pred=yc_cbow_pred))
print('========= Stance =========')
print(metrics.classification_report(y_true=ys_dev,y_pred=ys_cbow_pred))

In [ ]:
yc_sg_pred=SVMmodel(Xtrain=train_features_sg,y_train=yc,X_test=test_features_sg)

In [ ]:
ys_sg_pred=SVMmodel(Xtrain=train_features_sg,y_train=ys,X_test=test_features_sg)

In [ ]:
classify_accuracy=metrics.accuracy_score(y_true=yc_dev,y_pred=yc_sg_pred)
stance_accuracy=metrics.accuracy_score(y_true=ys_dev,y_pred=ys_sg_pred)


print("The classification accuracy after training on svm on all features : ",classify_accuracy)
print("The stance accuracy after training on svm on all features : ",stance_accuracy)

# F1 score
print('========= Category =========')
print(metrics.classification_report(y_true=yc_dev,y_pred=yc_sg_pred))
print('========= Stance =========')
print(metrics.classification_report(y_true=ys_dev,y_pred=ys_sg_pred))

In [ ]:
# features: bow, tfidf, cbow
yc_all_pred=SVMmodel(Xtrain=train_features,y_train=yc,X_test=test_features)

In [ ]:
# features: bow, tfidf, cbow
ys_all_pred=SVMmodel(Xtrain=train_features,y_train=ys,X_test=test_features)

In [ ]:
# classify_accuracy=metrics.accuracy_score(y_true=yc_dev,y_pred=yc_pred)
# stance_accuracy=metrics.accuracy_score(y_true=ys_dev,y_pred=ys_pred)

# classify_accuracy=metrics.accuracy_score(y_true=yc_dev,y_pred=yc_cbow_pred)
# stance_accuracy=metrics.accuracy_score(y_true=ys_dev,y_pred=ys_cbow_pred)

classify_accuracy=metrics.accuracy_score(y_true=yc_dev,y_pred=yc_all_pred)
stance_accuracy=metrics.accuracy_score(y_true=ys_dev,y_pred=ys_all_pred)


print("The classification accuracy after training on svm on all features : ",classify_accuracy)
print("The stance accuracy after training on svm on all features : ",stance_accuracy)

In [ ]:
# F1 score
print('========= Category =========')
print(metrics.classification_report(y_true=yc_dev,y_pred=yc_all_pred))
print('========= Stance =========')
print(metrics.classification_report(y_true=ys_dev,y_pred=ys_all_pred))

In [ ]:
# features: bow, tfidf, cbow, sg
yc_all_pred1=SVMmodel(Xtrain=train_features1,y_train=yc,X_test=test_features1)

In [ ]:
# features: bow, tfidf, cbow, sg
ys_all_pred1=SVMmodel(Xtrain=train_features1,y_train=ys,X_test=test_features1)

In [ ]:

classify_accuracy1=metrics.accuracy_score(y_true=yc_dev,y_pred=yc_all_pred1)
stance_accuracy1=metrics.accuracy_score(y_true=ys_dev,y_pred=ys_all_pred1)


print("The classification accuracy after training on svm on all features : ",classify_accuracy1)
print("The stance accuracy after training on svm on all features : ",stance_accuracy1)

In [ ]:
# F1 score
print('========= Category =========')
print(metrics.classification_report(y_true=yc_dev,y_pred=yc_all_pred))
print('========= Stance =========')
print(metrics.classification_report(y_true=ys_dev,y_pred=ys_all_pred))

In [ ]:
# F1 score
print('========= Category =========')
print(metrics.classification_report(y_true=yc_dev,y_pred=yc_all_pred))
print('========= Stance =========')
print(metrics.classification_report(y_true=ys_dev,y_pred=ys_all_pred))

In [ ]:
from torch import nn
# def initLSTM():
vocab_size = len(vocab)
embedding_dim = 100 #need to be tuned
hidden_size = 50 #need to be tuned

embedding = nn.Embedding(vocab_size, embedding_dim)

lstm = nn.LSTM(embedding_dim, hidden_size, batch_first=True)

n_classes_category = 9
n_classes_stance = 3

linear_category = nn.Linear(hidden_size, n_classes_category)
linear_stance = nn.Linear(hidden_size, n_classes_stance)


In [ ]:
def forward(self, sentences):
    """
    This function does the forward pass of our model
    Inputs:
    - sentences: tensor of shape (batch_size, max_length)

    Returns:
    - final_output: tensor of shape (batch_size, max_length, n_classes)
    """

    final_output = None
    ######################### TODO: implement the forward pass ####################################
    # (1) Pass the sentences through the embedding layer
    # (2) Pass the output of the embedding layer through the LSTM layer
    # (3) Pass the output of the LSTM layer through the linear layer
    # (4) Apply softmax to the output of the linear layer
    # (5) Return the output of the softmax layer
    sentences = embedding(sentences)
    sentences, _ = lstm(sentences)
    sentences_category = linear_category(sentences)
    sentences_stance = linear_stance(sentences)
    final_output_category = sentences_category
    final_output_stance = sentences_stance
    ###############################################################################################
    return final_output_category, final_output_stance

In [ ]:
# from keras.models import Sequential
# MAX_SEQ_LEN = 1000

# def model_1():
#     model = Sequential()
#     model.add(Embedding(input_dim = (len(tokenizer.word_counts) + 1), output_dim = 128, input_length = MAX_SEQ_LEN))
#     model.add(LSTM(128))
#     model.add(Dense(64, activation='relu'))
#     model.add(Dense(3, activation='softmax'))
#     model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
#     return model

# m1 = train(model_1, 
#            train_text_vec,
#            y_train,
#            test_text_vec,
#            y_test,
#            checkpoint_path='model_1.h5',
#            class_weights=cws
#           )